<a href="https://colab.research.google.com/github/efearase/RL_with_sentiment/blob/main/pulse_parsin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from httpx import ReadTimeout
from tpulse import TinkoffPulse
from httpx import HTTPStatusError
import time
import os
from IPython.display import clear_output
from pprint import pp
from dostoevsky.tokenization import RegexTokenizer
from dostoevsky.models import FastTextSocialNetworkModel

In [ ]:
names = pd.read_csv('Export_ru_securities-list_20240129.csv', encoding='windows-1251', delimiter=';', quotechar='"')

In [ ]:
codes = names[((names['LIST_SECTION']=='Первый уровень')) & (names['SUPERTYPE']=='Акции')]['TRADE_CODE'].dropna()

In [ ]:
date = pd.to_datetime(time.time(), unit='s')
date

Timestamp('2024-02-29 17:14:02.537048064')

In [ ]:
def datareader_optimized(ticker):
    cursor = 999999999
    t_0 = time.time()
    KEYS = ["id", "nickname", "content", "inserted"]
    date = pd.to_datetime(time.time(), unit='s').tz_localize('UTC')
    fin_date = pd.to_datetime('2022-03-29').tz_localize('UTC')
    pulse = TinkoffPulse()  # Assuming this is defined elsewhere

    # Initialize an empty DataFrame with specified types if possible
    base_df = pd.DataFrame(columns=["id", "nickname", "text", "inserted"])

    while date >= fin_date:
        try:
            if time.time() - t_0 > 1:
                print(f"{date}, {ticker}")
                clear_output(wait=True)
                t_0 = time.time()
            response = pulse.get_posts_by_ticker(ticker, cursor)
            cursor = response["nextCursor"]
            posts = response["items"]

            chunk_data = []
            for post in posts:
                data = {key: post[key] for key in KEYS if key in post}
                data['text'] = data['content'].get('text', None) if isinstance(data.get('content'), dict) else None
                data['inserted'] = pd.to_datetime(data['inserted']).strftime('%Y-%m-%d')
                del data['content']
                chunk_data.append(data)
                date = pd.to_datetime(post['inserted'])
            if chunk_data:
                chunk_df = pd.DataFrame(chunk_data)
                base_df = pd.concat([base_df, chunk_df], ignore_index=True)

        except (HTTPStatusError, TimeoutError, KeyError, ReadTimeout, NameError, TypeError) as e:
            print(f"Encountered an error: {e}. Continuing...")
            date = pd.to_datetime(date.timestamp() - 24 * 60 * 60, unit = 's').tz_localize('UTC')
            pass

        time.sleep(0.25)

    base_df.to_csv(f"data/text_{ticker}.csv", index=False)
    users = base_df['nickname'].unique()

    return users

In [ ]:
def check(name):
    for file in os.listdir('data'):
        if name in file:
            return 1
    return 0

In [ ]:
users_set = set()  # Use a set to automatically handle unique entries
for code in codes:
    if not check(code):  # Assuming check is a function defined elsewhere
        new_users = datareader_optimized(code)  # Collect new users from the optimized function
        users_set.update(new_users)  # Efficiently add new users, automatically avoiding duplicates

users = list(users_set)

2022-03-30 09:16:56.240000+03:00, TATNP


In [ ]:
user_info = pulse.get_user_info(users[0])
print(user_info['followersCount'])
if 'statistics' in user_info.keys():
    print(user_info['statistics']['yearRelativeYield'])

NameError: name 'pulse' is not defined

In [ ]:
retries = 5
yields = []
followers = []

for attempt in range(retries):
    try:
        for user in users:
            user_info = pulse.get_user_info(user)
            followers.append(user_info['followersCount'])
            if 'statistics' in user_info.keys():
                yields.append(user_info['statistics']['yearRelativeYield'])

    except ReadTimeout or ValueError or TimeoutError:
        print(f"Request timed out. Attempt {attempt+1} of {retries}. Retrying...")
        if attempt == retries - 1:
            print("Final attempt failed. Moving on to the next ticker...")
            pass
        time.sleep(some_seconds)
    except Exception as e:
        print(f"An error occurred: {e}")
        break

users_info = pd.DataFrame({'users': users, 'yields': yields, 'followers': followers})

users_info.to_csv("users_info.csv")

In [ ]:
users = base_df['nickname'].unique()

In [ ]:
followers = []
yields = []
users = base_df['nickname'].unique()

for user in users:
    user_info = pulse.get_user_info(user)

    followers.append(user_info['followersCount'])
    yields.append(user_info['statistics']['yearRelativeYield'])

users_info = pd.DataFrame({'users': users, 'yields': yields, 'followers', followers})

In [ ]:
user_info['statistics']['yearRelativeYield']